In [2]:
from Architectures.GAN import GAN
from Data.load import load
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
train, test = load("Pre-processed", take=100)

Loading images:  79%|███████▉  | 79/100 [00:33<00:08,  2.39it/s]

In [ ]:
gan_model = GAN(latent_dim=100)

NameError: name 'GAN' is not defined

In [ ]:
gan_model.train(train, epochs=10000, batch_size=128)

In [ ]:
latent_space = gan_model.encode(test)
decoded_image = gan_model.decode(latent_space)

print("Original image shape:", test.shape)
print("Latent space shape:", latent_space.shape)
print("Decoded image shape:", decoded_image.shape)